# Análisis exploratorio de datos del suministro de agua de la CDMX

Análisis exploratorio de datos del suministro de agua de la Ciudad de México a septiembre de 2020. Este análisis fue efectuado empleando la información bimestral por el concepto de suministro de agua a nivel manzana, considerando la facturación por servicio de consumo medido y promedio otorgada por el Gobierno de la Ciudad de México y disponible [en el portal de Datos Abiertos Ciudad de México](https://datos.cdmx.gob.mx/explore/dataset/consumo-agua/information/). 

Este análisis fue efecutado por el equipo de Ciencia de Datos conformado por:
* Carlos Bautista (125761)
* Enrique Ortiz (150644)
* Mario Heredia (197863)
* José Antonio Lechuga (192610)

## Contenido

En el presente reporte se abordan las siguientes preguntas y actividades:
* Exploración inicial
    * Generación de código exploratorio
    * ¿Cuántas variables existen?
    * ¿Cuántas observaciones existen?
    * ¿Cuántas observaciones únicas se tienen por variable?
    * ¿Cuántas variables numéricas existen?
    * ¿Cuántas variables de fecha existen?
    * ¿Cuántas variables categóricas existen?
    * ¿Cuántas variables de texto hay?
    * Generación del profiling de cada variable.
    * ¿Qué se conoce hasta ahora de este set de datos por variable?
    * ¿Cuántas alcadías hay? ¿Cuántos `nomgeo` existen? ¿Se identifica algún error?
* Limpieza y manipulación de datos
    * Generación de código de limpieza y manipulación
    * Transformación del nombre de las columnas a formato estándar: minúsculas, sin espacios en blanco (cambiar por guiones bajos) y sin signos de puntuación.
    * Agregar la variable `latitud` y `longitud` generadas a partir de la columna `geo_point`.
    * Transformar la variable latitud y longitud a numérica.
    * Eliminar la columna `geo_point`.
    * Eliminar la columna `geo_shape`.
    * Cambiar a minúsculas las columnas `alcaldia`, `colonia` e `indice_des`.
    * Efectuar nuevamente el proceso de identificación de variables numéricas, categóricas, texto y fechas.
    * Generación del profiling de cada variable.

## Exploración inicial

### Generación de código exploratorio

In [1]:
# verificando versiones
import datetime
import pandas as pd
import sys

from pandas_profiling import ProfileReport
sys.version

'3.7.4 (default, Sep 21 2020, 23:36:29) \n[Clang 11.0.0 (clang-1100.0.33.17)]'

In [2]:
def numero_variables_observaciones(data):
    observaciones, variables = data.shape
    print(f"Tenemos {observaciones} observaciones para {variables} variables.")
    
def contar_variables_numericas(data):
    # con 'data.dtypes' podemos ver todos los tipos de variables 
    numeric = [e for e in data.dtypes.values if e == "float64" or e == "int64"]
    print(f"Tenemos {len(numeric)} variables numéricas.")
    
def contar_variables_fecha(data):   
    date_variables = [e for e in data.dtypes.values if type(e) is datetime.date]
    print(f"Tenemos {len(date_variables)} variables de fecha.")
    
def generar_data_profiling(title, path):
    profile = ProfileReport(data, title=title)
    profile.to_file(output_file=path)
    return profile

def numero_valores_unicos(data, column):
    if column not in data.columns:
        print("Nombre de columna invalido")
    else:
        return len(data[column].unique())

In [3]:
# cambiar a donde se encuentre el archivo csv con los datos requeridos
data_path = 'datos/consumo-agua.csv'

In [4]:
data = pd.read_csv(data_path)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

### ¿Cuántas variables existen? ¿Cuántas observaciones existen?

In [5]:
numero_variables_observaciones(data)

Tenemos 71102 observaciones para 17 variables.


In [6]:
data.head()

,Geo Point,Geo Shape,consumo_total_mixto,anio,nomgeo,consumo_prom_dom,consumo_total_dom,alcaldia,colonia,consumo_prom_mixto,consumo_total,consumo_prom,consumo_prom_no_dom,bimestre,consumo_total_no_dom,gid,indice_des
0,"19.4552601937,-99.1126617526","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",159.72,2019,Gustavo A. Madero,42.566364,468.23,GUSTAVO A. MADERO,7 DE NOVIEMBRE,53.24,631.00,42.066667,3.05,3,3.05,57250,ALTO
1,"19.4552601937,-99.1126617526","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,35.936667,107.81,GUSTAVO A. MADERO,7 DE NOVIEMBRE,0.00,115.13,28.782500,7.32,3,7.32,57253,MEDIO
2,"19.4557195871,-99.1135822797","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,24.586000,122.93,GUSTAVO A. MADERO,7 DE NOVIEMBRE,0.00,197.96,32.993333,75.03,3,75.03,57255,POPULAR
3,"19.4596467168,-99.1044693641","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,0.000000,0.00,GUSTAVO A. MADERO,NUEVA TENOCHTITLAN,0.00,253.53,84.510000,84.51,3,253.53,57267,BAJO
4,"19.4741606185,-99.1467497317","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",56.72,2019,Azcapotzalco,67.436250,539.49,AZCAPOTZALCO,PROHOGAR,56.72,839.35,76.304545,121.57,3,243.14,57330,BAJO


### ¿Cuántas observaciones únicas se tienen por variables? 


In [7]:
data.nunique()

Geo Point               22930
Geo Shape               22922
consumo_total_mixto     24339
anio                        1
nomgeo                     17
consumo_prom_dom        52060
consumo_total_dom       47051
alcaldia                   16
colonia                  1340
consumo_prom_mixto      31911
consumo_total           56015
consumo_prom            62214
consumo_prom_no_dom     37440
bimestre                    3
consumo_total_no_dom    27336
gid                     71102
indice_des                  4
dtype: int64

### ¿Cuántas variables numéricas existen? 

In [8]:
# podemos ver todos los tipos de variables
data.dtypes

Geo Point                object
Geo Shape                object
consumo_total_mixto     float64
anio                      int64
nomgeo                   object
consumo_prom_dom        float64
consumo_total_dom       float64
alcaldia                 object
colonia                  object
consumo_prom_mixto      float64
consumo_total           float64
consumo_prom            float64
consumo_prom_no_dom     float64
bimestre                  int64
consumo_total_no_dom    float64
gid                       int64
indice_des               object
dtype: object

In [9]:
contar_variables_numericas(data)

Tenemos 11 variables numéricas.


### ¿Cuántas variables de fecha tenemos? 

Ninguna. Se puede ver en la impresión de dtypes. La variable `anio` indica el año, pero eso no es suficiente para ser una fecha (no califica como date, time o datetime). 

In [10]:
contar_variables_fecha(data)

Tenemos 0 variables de fecha.


### ¿Cuántas variables categóricas existen? 

Las siguientes pueden ser consideradas variables categóricas: `nomgeo`, `alcaldia`, `colonia`, e `indice_des`. `bimestre` fue contada como numérica, pero es cierto que puede ser analizada como categórica, al igual que año (dado que solo hay cifras para 2019). Por lo tanto, tenemos entre 4 y 6 variables categóricas, dependiendo de si tomamos a bimestre y año como categórica o no.  

### ¿Cuántas variables de texto hay? 

Aunque `nomgeo`, `alcaldia`, `colonia` e `indice_des` son texto, no contienen como tal texto a analizar (como palabras u oraciones). Diríamos que son categóricas.

### Generación del profiling de cada variable

In [11]:
initial_profile = generar_data_profiling('Profiling inicial de consumo de agua', 'reportes/profiling_inicial.html')

### ¿Qué se conoce hasta ahora de este set de datos por variable?

* `Geo Point` indica la latitud y longitud de la colonia. 
* `Geo Shape` es un diccionario con información geográfica. 
* `nomgeo` y `alcaldia` son equivalentes (e indican el nombre de la alcaldía). Hay 16 distintos. 
* `colonia` indica el nombre de la colonia.
* `anio` indica el año, y solo hay datos para 2019.
* `bimestre` indica el bimestre del año, y solo hay datos para el bimestre 1, 2 y 3. 

### ¿Cuántas alcaldías hay? ¿Cuántos `nomgeo` existen? ¿Se identifica algún error? 

In [12]:
no_alcaldias = numero_valores_unicos(data, 'alcaldia')
no_nomgeo = numero_valores_unicos(data, 'nomgeo')
print(f'Se tiene {no_alcaldias} alcaldías únicas y {no_nomgeo} nomgeo únicos.')

Se tiene 16 alcaldías únicas y 17 nomgeo únicos.


Sí el error está en la variable nomgeo: se registró "Talpan" en vez de "Tlalpan". 

In [13]:
[e for e in data.nomgeo.unique() if e.endswith('pan')]

['Talpan', 'Tlalpan']

## Limpieza y manipulación de datos

### Generación de código de limpieza y manipulación

In [14]:
def standarize_column_names(data, excluded_punctuation=".,-*¿?¡!"):
    data.columns = data.columns.str.lower().str.replace(" ", "_")
    for ch in excluded_punctuation:
        data.columns = data.columns.str.replace(ch, "")
        
        
def standarize_column_strings(data, columns, excluded_punctuation=".,-*¿?¡!"):
    for col in columns:
        data[col] = data[col].apply(lambda x: x.lower().replace(" ", "_"))
        for ch in excluded_punctuation:
            data[col] = data[col].apply(lambda x: x.replace(ch, ""))
    
def replace_column_value(data, column, search_string, replace_string):
    data[column] = data[column].apply(lambda x: x.replace(search_string, replace_string))
    

def clean_data_from(path):
    """
    Función para realizar todas las transformaciones necesarias requeridas al 
    dataset del consumo de agua. 
    
    Inputs:
    -------
    
    path: str
          El path en tu compu donde está el archivo csv. ("/Users/etc/etc..")
    
    Returns:
    --------
    
    data: pandas dataframe
          Un dataframe en el formato correcto
    
    """
    #1. cargar la info y arreglar nombres de columnas
    data = pd.read_csv(path)
    standarize_column_names(data)
    
    #2. Transformar alcaldia, colonia e indice_des
    col_text = ["alcaldia", "colonia", "indice_des"]
    standarize_column_strings(data, col_text)
    
    #3. Agregar latitud y longitud en formato numérico
    latitud = [float(e.split(",")[0]) for e in data["geo_point"]] 
    longitud = [float(e.split(",")[1]) for e in data["geo_point"]]
    data["latitud"] = latitud 
    data["longitud"] = longitud
    
    #4. Tirar variables irrelevantes
    data.drop(columns = ["geo_point", "geo_shape"], inplace = True)
    
    #5. Quitar el error de Talpan
    replace_column_value(data, 'nomgeo', 'Talpan', 'Tlalpan')
    
    return data

### Transformación del nombre de las columnas a formato estándar

In [15]:
standarize_column_names(data)
data.head()

,geo_point,geo_shape,consumo_total_mixto,anio,nomgeo,consumo_prom_dom,consumo_total_dom,alcaldia,colonia,consumo_prom_mixto,consumo_total,consumo_prom,consumo_prom_no_dom,bimestre,consumo_total_no_dom,gid,indice_des
0,"19.4552601937,-99.1126617526","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",159.72,2019,Gustavo A. Madero,42.566364,468.23,GUSTAVO A. MADERO,7 DE NOVIEMBRE,53.24,631.00,42.066667,3.05,3,3.05,57250,ALTO
1,"19.4552601937,-99.1126617526","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,35.936667,107.81,GUSTAVO A. MADERO,7 DE NOVIEMBRE,0.00,115.13,28.782500,7.32,3,7.32,57253,MEDIO
2,"19.4557195871,-99.1135822797","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,24.586000,122.93,GUSTAVO A. MADERO,7 DE NOVIEMBRE,0.00,197.96,32.993333,75.03,3,75.03,57255,POPULAR
3,"19.4596467168,-99.1044693641","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,0.000000,0.00,GUSTAVO A. MADERO,NUEVA TENOCHTITLAN,0.00,253.53,84.510000,84.51,3,253.53,57267,BAJO
4,"19.4741606185,-99.1467497317","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",56.72,2019,Azcapotzalco,67.436250,539.49,AZCAPOTZALCO,PROHOGAR,56.72,839.35,76.304545,121.57,3,243.14,57330,BAJO


### Agregar la variable latitud y longitud generadas a partir de `geopoint`. Transformar la variable latitud y longitud a numérica.

In [16]:
# sin lo de "float" lo toma como "object"
latitud = [float(e.split(",")[0]) for e in data["geo_point"]] 
longitud = [float(e.split(",")[1]) for e in data["geo_point"]]
 
data["latitud"] = latitud 
data["longitud"] = longitud

data.dtypes

geo_point                object
geo_shape                object
consumo_total_mixto     float64
anio                      int64
nomgeo                   object
consumo_prom_dom        float64
consumo_total_dom       float64
alcaldia                 object
colonia                  object
consumo_prom_mixto      float64
consumo_total           float64
consumo_prom            float64
consumo_prom_no_dom     float64
bimestre                  int64
consumo_total_no_dom    float64
gid                       int64
indice_des               object
latitud                 float64
longitud                float64
dtype: object

### Eliminar la columna `geo_point` y `geo_shape`

In [17]:
data.drop(columns = ["geo_point", "geo_shape"], inplace = True)
data.head()

,consumo_total_mixto,anio,nomgeo,consumo_prom_dom,consumo_total_dom,alcaldia,colonia,consumo_prom_mixto,consumo_total,consumo_prom,consumo_prom_no_dom,bimestre,consumo_total_no_dom,gid,indice_des,latitud,longitud
0,159.72,2019,Gustavo A. Madero,42.566364,468.23,GUSTAVO A. MADERO,7 DE NOVIEMBRE,53.24,631.00,42.066667,3.05,3,3.05,57250,ALTO,19.455260,-99.112662
1,0.00,2019,Gustavo A. Madero,35.936667,107.81,GUSTAVO A. MADERO,7 DE NOVIEMBRE,0.00,115.13,28.782500,7.32,3,7.32,57253,MEDIO,19.455260,-99.112662
2,0.00,2019,Gustavo A. Madero,24.586000,122.93,GUSTAVO A. MADERO,7 DE NOVIEMBRE,0.00,197.96,32.993333,75.03,3,75.03,57255,POPULAR,19.455720,-99.113582
3,0.00,2019,Gustavo A. Madero,0.000000,0.00,GUSTAVO A. MADERO,NUEVA TENOCHTITLAN,0.00,253.53,84.510000,84.51,3,253.53,57267,BAJO,19.459647,-99.104469
4,56.72,2019,Azcapotzalco,67.436250,539.49,AZCAPOTZALCO,PROHOGAR,56.72,839.35,76.304545,121.57,3,243.14,57330,BAJO,19.474161,-99.146750


### Cambiar a minúsculas las columnas alcaldía, colonia e indice_des.

In [18]:
col_text = ["alcaldia", "colonia", "indice_des"]
standarize_column_strings(data, col_text)
data.head()

,consumo_total_mixto,anio,nomgeo,consumo_prom_dom,consumo_total_dom,alcaldia,colonia,consumo_prom_mixto,consumo_total,consumo_prom,consumo_prom_no_dom,bimestre,consumo_total_no_dom,gid,indice_des,latitud,longitud
0,159.72,2019,Gustavo A. Madero,42.566364,468.23,gustavo_a_madero,7_de_noviembre,53.24,631.00,42.066667,3.05,3,3.05,57250,alto,19.455260,-99.112662
1,0.00,2019,Gustavo A. Madero,35.936667,107.81,gustavo_a_madero,7_de_noviembre,0.00,115.13,28.782500,7.32,3,7.32,57253,medio,19.455260,-99.112662
2,0.00,2019,Gustavo A. Madero,24.586000,122.93,gustavo_a_madero,7_de_noviembre,0.00,197.96,32.993333,75.03,3,75.03,57255,popular,19.455720,-99.113582
3,0.00,2019,Gustavo A. Madero,0.000000,0.00,gustavo_a_madero,nueva_tenochtitlan,0.00,253.53,84.510000,84.51,3,253.53,57267,bajo,19.459647,-99.104469
4,56.72,2019,Azcapotzalco,67.436250,539.49,azcapotzalco,prohogar,56.72,839.35,76.304545,121.57,3,243.14,57330,bajo,19.474161,-99.146750


### Efectuar nuevamente el proceso de identificación de variables numéricas, categóricas, texto y fechas

* **Variables numéricas:** 
    * `consumo_total_mixto`
    * `anio` (de nuevo, que puede ser contada como categórica)
    * `consumo_promo_dom`
    * `consumo_total_dom`
    * `consumo_prom_mixto`
    * `consumo_total`
    * `consumo_prom`
    * `consumo_prom_no_dom`
    * `consumo_total_no_dom`
    * `gid`
    * `bimestre` (una vez mas) puede ser o no contada como numérica

* **Variables de fecha:** ninguna. 

* **Variables de texto:** 
    * `latitud` y `longitud` son texto para pandas, aunque en realidad son coordenadas
    * No hay variables que contengan texto a analizar

* **Variables categóricas:** 
    * `nomgeo`
    * `alcaldia`
    * `colonia`
    * `indices_des`
    * `bimestre`

In [19]:
contar_variables_numericas(data)
contar_variables_fecha(data)

Tenemos 13 variables numéricas.
Tenemos 0 variables de fecha.


In [20]:
replace_column_value(data, 'nomgeo', 'Talpan', 'Tlalpan')
data.nomgeo.unique()

array(['Gustavo A. Madero', 'Azcapotzalco', 'Coyoacán', 'Tlalpan',
       'Milpa Alta', 'Álvaro Obregón', 'Cuajimalpa de Morelos', 'Tláhuac',
       'La Magdalena Contreras', 'Xochimilco', 'Iztapalapa', 'Iztacalco',
       'Benito Juárez', 'Miguel Hidalgo', 'Cuauhtémoc',
       'Venustiano Carranza'], dtype=object)

In [21]:
data.nomgeo.nunique()

16

### Genera el data profiling por variable. 

In [22]:
final_profile = generar_data_profiling('Profiling final de consumo de agua', 'reportes/profiling_final.html')

In [23]:
# prueba de funcion de limpieza global
clean_data = clean_data_from(data_path)
clean_data.head()

,consumo_total_mixto,anio,nomgeo,consumo_prom_dom,consumo_total_dom,alcaldia,colonia,consumo_prom_mixto,consumo_total,consumo_prom,consumo_prom_no_dom,bimestre,consumo_total_no_dom,gid,indice_des,latitud,longitud
0,159.72,2019,Gustavo A. Madero,42.566364,468.23,gustavo_a_madero,7_de_noviembre,53.24,631.00,42.066667,3.05,3,3.05,57250,alto,19.455260,-99.112662
1,0.00,2019,Gustavo A. Madero,35.936667,107.81,gustavo_a_madero,7_de_noviembre,0.00,115.13,28.782500,7.32,3,7.32,57253,medio,19.455260,-99.112662
2,0.00,2019,Gustavo A. Madero,24.586000,122.93,gustavo_a_madero,7_de_noviembre,0.00,197.96,32.993333,75.03,3,75.03,57255,popular,19.455720,-99.113582
3,0.00,2019,Gustavo A. Madero,0.000000,0.00,gustavo_a_madero,nueva_tenochtitlan,0.00,253.53,84.510000,84.51,3,253.53,57267,bajo,19.459647,-99.104469
4,56.72,2019,Azcapotzalco,67.436250,539.49,azcapotzalco,prohogar,56.72,839.35,76.304545,121.57,3,243.14,57330,bajo,19.474161,-99.146750


## Exposición 2

### Data Profiling

**Datos faltantes y completos**

**Comportamiento del consumo bimestral**

No observamos diferencia significativa en el consumo por bimestre (justificar) ...

### Consumo total

Por alcaldía (gráfica de barras ordenada de mayor a menor y descripción breve)

Por índice de desarrollo (boxplot y descripción breve))

Participación de cada tipo de consumo (doméstico, no doméstico y mixto) en el total (puede ser gráf o tabla)

### Consumo promedio

Por alcaldía (gráfica de barras ordenada de mayor a menor y descripción breve)

Por índice de desarrollo (boxplot y descripción breve))

Participación de cada tipo de consumo (doméstico, no doméstico y mixto) en el total (puede ser gráf o tabla)

### Distribución de las colonias de la CDMX

Por alcaldía (conteo y descripción de qué alcaldía tiene más colonias, etc.)

Por índice de desarrollo (conteo y descripción de colonias según idh)

### Características de los datos para la definición de un modelo

Argumentos: consideramos que información no es suficiente ni completa para predecir un modelo correcto. Nos gustaría contar con información adicional como número de tomas (para controlar), tarifa promedio, entre otros...

Variables que nos servirían para predecir idh: consumo promedio, tipo de consumo y su proporción, ubicación (alcaldía).